# Monthly Retrospective script
The following script will produce a list of statistics that might be relevant for the Monthly retrospective. **This will only capture earthquakes > 7 days ago. As such I reccommend waiting until the 8th onwards to run it**.

**To run this script properly, first please restart the kernal to wipe the cache, else it will crash. You can do this by clicking the restart button in the top left, or by right clicking anywhere on the script.**

Once this is done, please adjust the values in the "Times" section below as per instructions, then run all cells like you would a volcano script. 

This is a work in progress and would appreciate any feedback as to what other information, graphs etc would be useful. 

In [1]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy import read_inventory
import obspy
client = Client("GEONET") #yolo
from obspy import UTCDateTime

## Times
Unfortunately the kernel appears to crash if I try to bring a whole month's worth of data in at once. As such I've split it into a bunch of little parts then added them together. 

This means the user must change the last day of the previous month in the "startdate" variable (be sure to check for 30 vs 31 days), and the month you're doing the restrospective for in the for loop (where it says times.append). Then, change the number in the "range" function to be the number of days in that particular month **minus 1** (so, 29 for June, 30 for august).

This corresponds to midnight on the first of the month, to midnight on the first of the next month in NZT, as Obspy runs using UTC time. 

For example, for August you'd have the following:

![alt tex](Image3.PNG "Example")

I've also currently got it set to daylight savings time - outside of daylight savings the "11:00:00" part should be "12:00:00". 

This makes the code very inefficient but for high activity months like June the kernel would simply crash every time so it's the only way I can make it work at the moment. If it still doesn't work, try downloading it, importing it into Spyder and running it that way, though that will require Obspy to be installed on that laptop - see instructions in Geohazard Analysys/other projects/obspy guide and examples. 

In [2]:
startdate = "2019-06-30 11:00:00"
times = []
for a in range(30):
    times.append("2019-07-"+str(a+1)+" 11:00:00")

The following section is the code that will automatically run with the above input. The output is found at the bottom.

In [3]:
cat = obspy.core.event.Catalog()
for a in range(len(times)):
    cat+=client.get_events(starttime = times[a], endtime = UTCDateTime(times[a]) + 86400)
cat+=client.get_events(starttime = startdate, endtime = UTCDateTime(startdate) + 86400)

In [4]:
LessOne = 0
OneTwo = 0
TwoThree = 0
ThreeFour = 0
FourFive = 0
FiveSix = 0
SixPlus = 0

In [5]:
for a in range(len(cat)):
    for b in range(len(cat[a].magnitudes)):
        if cat[a].preferred_magnitude_id == cat[a].magnitudes[b].resource_id and  cat[a].magnitudes[b].mag <= 1:
            LessOne = LessOne + 1
        if cat[a].preferred_magnitude_id == cat[a].magnitudes[b].resource_id and  cat[a].magnitudes[b].mag >= 1 and cat[a].magnitudes[b].mag <= 2:
            OneTwo = OneTwo + 1
        if cat[a].preferred_magnitude_id == cat[a].magnitudes[b].resource_id and  cat[a].magnitudes[b].mag >= 2 and cat[a].magnitudes[b].mag <= 3:
            TwoThree = TwoThree + 1
        if cat[a].preferred_magnitude_id == cat[a].magnitudes[b].resource_id and  cat[a].magnitudes[b].mag >= 3 and cat[a].magnitudes[b].mag <= 4:
            ThreeFour = ThreeFour + 1
        if cat[a].preferred_magnitude_id == cat[a].magnitudes[b].resource_id and  cat[a].magnitudes[b].mag >= 4 and cat[a].magnitudes[b].mag <= 5:
            FourFive = FourFive + 1
        if cat[a].preferred_magnitude_id == cat[a].magnitudes[b].resource_id and  cat[a].magnitudes[b].mag >= 5 and cat[a].magnitudes[b].mag <= 6:
            FiveSix = FiveSix + 1    
        if cat[a].preferred_magnitude_id == cat[a].magnitudes[b].resource_id and  cat[a].magnitudes[b].mag >= 6:
            SixPlus = SixPlus + 1
print("<1:", LessOne,"\n", "1-2:", OneTwo,"\n","2-3:", TwoThree,"\n", "3-4:", ThreeFour,"\n", "4-5:", FourFive,"\n", "5-6:", FiveSix,"\n", "6+:", SixPlus)
local = 0
kermadec = 0
tele = 0
for a in range(len(cat)):
    for b in range(len(cat[a].origins)):
        if cat[a].origins[b].latitude <= -33 and cat[a].origins[b].latitude >= -50 and cat[a].origins[b].longitude >= 163 and cat[a].origins[b].longitude <=180 or cat[a].origins[b].latitude <= -33 and cat[a].origins[b].latitude >= -50 and cat[a].origins[b].longitude <=-175 and cat[a].origins[b].longitude >= -180 or cat[a].origins[b].latitude <= -32 and cat[a].origins[b].latitude >= -33 and cat[a].origins[b].longitude <=177 and cat[a].origins[b].longitude >= 163:
            local = local + 1
        if cat[a].origins[b].latitude <= -25 and cat[a].origins[b].latitude >= -33 and cat[a].origins[b].longitude >= 177 and cat[a].origins[b].longitude <=180 or cat[a].origins[b].latitude <= -25 and cat[a].origins[b].latitude >= -33 and cat[a].origins[b].longitude <=-169 and cat[a].origins[b].longitude >= -180:
            kermadec = kermadec + 1       
tele = len(cat) - local - kermadec
print("local",local,"\n","kermadec",kermadec,"\n","tele",tele)
MaxMag = list()
for a in range(len(cat)):
    for b in range(len(cat[a].magnitudes)):
        if cat[a].preferred_magnitude_id == cat[a].magnitudes[b].resource_id:
            MaxMag.append(cat[a].magnitudes[b].mag)
x = max(MaxMag)
for a in range(len(cat)):
    for b in range(len(cat[a].magnitudes)):
        if cat[a].preferred_magnitude_id == cat[a].magnitudes[b].resource_id:
            if x == cat[a].magnitudes[b].mag:
                print("Maximum magnitude", max(MaxMag), cat[a].resource_id)
MinMag = list()
for a in range(len(cat)):
    for b in range(len(cat[a].magnitudes)):
        if cat[a].preferred_magnitude_id == cat[a].magnitudes[b].resource_id:
            MinMag.append(cat[a].magnitudes[b].mag)
x = min(MinMag)
for a in range(len(cat)):
    for b in range(len(cat[a].magnitudes)):
        if cat[a].preferred_magnitude_id == cat[a].magnitudes[b].resource_id:
            if x == cat[a].magnitudes[b].mag:
                print("Minimum magnitude", min(MinMag), cat[a].resource_id)

<1: 61 
 1-2: 830 
 2-3: 754 
 3-4: 120 
 4-5: 105 
 5-6: 58 
 6+: 8
local 1804 
 kermadec 64 
 tele 68
Maximum magnitude 7.145370493 smi:nz.org.geonet/2019p507127
Minimum magnitude 0.1224906838 smi:nz.org.geonet/2019p559041
